In [1]:
%%capture
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score

In [2]:
QPATH = "Quantlet/4-qode2desc"

In [3]:
import sys

IN_COLAB = "google.colab" in sys.modules

import os

if IN_COLAB:
    os.chdir(
        f"/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}"
    )
else:
    %load_ext lab_black

In [6]:
import pickle
import json
import re
import sys
from IPython.display import display
import datetime

from tqdm import tqdm
import pandas as pd
import numpy as np

tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelWithLMHead,
    SummarizationPipeline,
    AutoModelForSeq2SeqLM,
)
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import nltk

nltk.download("punkt")
import evaluate

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import importlib
import analysis_modules

importlib.reload(analysis_modules)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'analysis_modules' from '/usr/net/zinovyee.hub/IRTG/MLSC/Encode-the-Qode/Quantlet/4-qode2desc/analysis_modules.py'>

In [15]:
def batch_tokenize_preprocess(batch, tokenizer, max_input_length, max_output_length):

    source = batch["input_sequence"]
    target = batch["output_sequence"]

    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_input_length
    )

    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_output_length
    )

    batch = {k: v for k, v in source_tokenized.items()}

    # Ignore padding in the loss

    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]

    return batch

In [18]:
model_name = "Salesforce/codet5-base-multi-sum"

In [19]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)

In [45]:
tokenizer

RobertaTokenizerFast(name_or_path='Salesforce/codet5-base-multi-sum', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': [AddedToken("<extra_id_99>", rstrip=False, lstrip=True, single_word=False, normalized=True), AddedToken

In [20]:
train_dataset = load_dataset(
    "json",
    data_files="train_dataset_20231021_sample0.json",
    field="data",
    data_dir="/home/RDC/zinovyee.hub/H:/zinovyee.hub/IRTG/MLSC/Encode-the-Qode/data/preprocessed/Quantlet/20231021/no_context/",
)

Found cached dataset json (/home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-a9640bba95be2093/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
tokenized = train_data_txt.map(
    lambda batch: tokenizer(batch["input_sequence"], max_length=None)
)

Map:   0%|          | 0/3945 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1111 > 512). Running this sequence through the model will result in indexing errors


In [44]:
tokenizer.tokenize(train_data_txt["input_sequence"][100])

['#',
 'ĠClear',
 'Ġvariables',
 'Ġand',
 'Ġclose',
 'Ġwindows',
 'Ċ',
 'rm',
 '(',
 'list',
 'Ġ=',
 'Ġls',
 '(',
 'all',
 'Ġ=',
 'ĠTRUE',
 '))',
 'Ċ',
 'graphics',
 '.',
 'off',
 '()',
 'Ċ',
 '#',
 'ĠLoad',
 'Ġpackages',
 'Ċ',
 'require',
 '(',
 'd',
 'ply',
 'r',
 ')',
 'Ċ',
 'require',
 '(',
 'foreach',
 ')',
 'Ċ',
 'library',
 '(',
 'parallel',
 ')',
 'Ċ',
 'require',
 '(',
 'do',
 'MC',
 ')',
 'Ċ',
 'register',
 'Do',
 'MC',
 '(',
 'parallel',
 '::',
 'detect',
 'Cores',
 '())',
 'Ġ#',
 'ĠRegister',
 'Ġall',
 'Ġcores',
 'Ġfor',
 'Ġparallel',
 'ized',
 'Ġcomputation',
 'Ċ',
 '#',
 'Ġ-----------------------------------------------------------------',
 '----------------',
 '--------',
 'Ċ',
 '#',
 'ĠDefine',
 'Ġsimulation',
 'Ġfunction',
 'Ċ',
 'two',
 'Stage',
 'Sim',
 'Ġ=',
 'Ġfunction',
 '(',
 'n',
 '_',
 'obs',
 'Ġ=',
 'ĠNA',
 ',',
 'Ġcov',
 '_',
 'error',
 'Ġ=',
 'ĠNA',
 ',',
 'Ġbeta',
 '_',
 'z',
 '1',
 'Ġ=',
 'ĠNA',
 ',',
 'Ġbeta',
 '_',
 'z',
 '2',
 'Ġ=',
 'ĠNA',
 '){',
 'Ċ',

In [40]:
tokenized["input_sequence"][0]

"close all\ngraph = 1;\n% Open & read txt file with names of files to work with\nfilelist = 'filelist.txt';\nfilenames = textread(filelist,'%s');\nnfiles = length(filenames);\ndisp([filelist, ' has ', num2str(nfiles), ' files.']);\nfigcount = 0;\n% Processing txt files\nfor i = 1:nfiles\n% checking what kind of file (single or mapping) it is\nfilename = char(filenames(i));\nfprintf('%s: single file\\n', filename);\n[x,y] = textread(filename,'%f%f','headerlines',1); % Reads in txt file\ndata = [x y];\nFS = process(data, graph, filename);\nfigcount = figcount + 1;\n% pausing program to look at figures\nif graph == 1 && figcount > 50\nfprintf('PAUSED after displaying %d figures\\n\\n', figcount);\npause\nfigcount = 0;\nend\nend\n%------------------------------------------------------\n% Pre-processing data file\nfunction FS = process(data, graph, filename)\ndata = sortrows(data,1); % sort rows in ascending order\nrawdata = data;\nmedfiltdata(:,2) = medfilt1(data(:,2),5); % applies median 

In [21]:
train_data_txt = train_dataset["train"]
train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch,
        tokenizer=tokenizer,
        max_input_length=encoder_max_length,
        max_output_length=decoder_max_length,
    ),
    batch_size=8,
    batched=True,
    remove_columns=train_data_txt.column_names,
)

Map:   0%|          | 0/3945 [00:00<?, ? examples/s]

NameError: name 'encoder_max_length' is not defined

In [ ]:


                                
   
        
    validation_data_txt = test_dataset['train']
    
   
    
    validation_data = validation_data_txt.map(
        lambda batch: batch_tokenize_preprocess(
            batch, 
            tokenizer=tokenizer,
            max_input_length=encoder_max_length,
            max_output_length=decoder_max_length
        ),
        batched=True,
        remove_columns=validation_data_txt.column_names,
    )

In [ ]:
def train_analyze(analysis_name: str,
                    model_name: str, 
                    train_data_path: str, 
                    val_data_path:   str,
                    train_data_name: str,
                    val_data_name: str,
                    encoder_max_length: int, 
                    decoder_max_length: int,
                    random_state: int, 
                    learning_rate: float=5e-5,
                    epochs: int=4, 
                    train_batch: int=4, 
                    eval_batch: int=4,
                    warmup_steps: int=500, 
                    weight_decay: float=0.1,
                    logging_stes: int=100,
                    save_total_lim: int=3,
                    label_smooting: float = 0.1,
                    predict_generate: bool=True,
                    eval_columns_list: list=['eval_loss', 'eval_rouge1'],
                    save_strategy='no',
                    load_best_model_at_end=True,
                    evaluate_only=False,
                    report_to=None): 
                         
    # CREATE ANALYSIS FOLDER
    os.mkdir(f'analysis_report_{analysis_name}')
    print(analysis_name)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    
    if model_name=='CodeT5':
        model_name="Salesforce/codet5-base-multi-sum"
        
    elif model_name=='CodeTrans':
        model_name="SEBIS/code_trans_t5_base_source_code_summarization_python_multitask"
                         
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)
    model.to(device)
    print(device)
    
    train_dataset = load_dataset("json",
                             data_files=train_data_name,
                             field="data",
                             data_dir=train_data_path)

    test_dataset = load_dataset("json",
                                data_files=val_data_name,
                                field="data",
                                data_dir=val_data_path)
                                
    train_data_txt = train_dataset['train']
        
    validation_data_txt = test_dataset['train']
    
    train_data = train_data_txt.map(
        lambda batch: batch_tokenize_preprocess(
            batch, 
            tokenizer=tokenizer,
            max_input_length=encoder_max_length,
            max_output_length=decoder_max_length
        ),
        batch_size=8,
        batched=True,
        remove_columns=train_data_txt.column_names,
    )
    
    validation_data = validation_data_txt.map(
        lambda batch: batch_tokenize_preprocess(
            batch, 
            tokenizer=tokenizer,
            max_input_length=encoder_max_length,
            max_output_length=decoder_max_length
        ),
        batched=True,
        remove_columns=validation_data_txt.column_names,
    )
    
    
    # SUBSAMPLE FOR GENERATION BEFORE TUNING
    test_samples = validation_data_txt.select(range(20))
    summaries_before_tuning = generate_summary(test_samples, 
                                                model, 
                                                tokenizer, 
                                                encoder_max_length)[1]

    
    training_args = Seq2SeqTrainingArguments(
        output_dir=f"analysis_report_{analysis_name}/results",
        num_train_epochs=epochs,
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=train_batch,
        per_device_eval_batch_size=eval_batch,
        learning_rate=learning_rate,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        label_smoothing_factor=label_smooting,
        predict_with_generate=predict_generate,
        logging_dir=f"analysis_report_{analysis_name}/logs",
        logging_steps=logging_stes,
        save_total_limit=save_total_lim,
        report_to=report_to,
        save_strategy=save_strategy,
        load_best_model_at_end=load_best_model_at_end
    )
    
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    compute_metrics = compute_metric_with_params(tokenizer)
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_data,
        eval_dataset=validation_data,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    
    # ZERO - SHOT
    results_zero_shot = trainer.evaluate()
    results_zero_shot_df = pd.DataFrame(data=results_zero_shot, index=[0])[eval_columns_list]
    results_zero_shot_df.loc[0, :] = results_zero_shot_df.loc[0, :].apply(lambda x: round(x, 3))
    print(results_zero_shot_df)
    
    results_zero_shot_df.to_csv(f'analysis_report_{analysis_name}/results_zero_shot.csv', index=False)
    
    if evaluate_only:
        with open(f'analysis_report_{analysis_name}/results.txt', "w") as results_file:
        
            for i, description in enumerate(test_samples["output_sequence"]):
                results_file.write('_'*10)
                results_file.write(f'Original: {description}')
                results_file.write(f'Summaries: {summaries_before_tuning[i]}')
        return 'Finished'
        
    
    # TRAINING
    trainer.train()
    
    # FINE-TUNING
    results_fine_tune = trainer.evaluate()
    results_fine_tune_df = pd.DataFrame(data=results_fine_tune, index=[0])[eval_columns_list]
    results_fine_tune_df.loc[0, :] = results_fine_tune_df.loc[0, :].apply(lambda x: round(x, 3))
    print(results_fine_tune_df)
    
    results_fine_tune_df.to_csv(f'analysis_report_{analysis_name}/results_fine_tune.csv', index=False)
    
    summaries_after_tuning = generate_summary(test_samples, 
                                             model,
                                             tokenizer,
                                             encoder_max_length)[1]
    
    for i, description in enumerate(test_samples["output_sequence"]):
      print('_'*10)
      print(f'Original: {description}')
      print(f'Summary before Tuning: {summaries_before_tuning[i]}')
      print(f'Summary after Tuning: {summaries_after_tuning[i]}')
      print('_'*10)
      print('\n')
      
    # CREATE REPORT
    with open(f'analysis_report_{analysis_name}/results.txt', "w") as results_file:
        
        for i, description in enumerate(test_samples["output_sequence"]):
            results_file.write('_'*10)
            results_file.write(f'Original: {description}')
            results_file.write(f'Summary before Tuning: {summaries_before_tuning[i]}')
            results_file.write(f'Summary after Tuning: {summaries_after_tuning[i]}')
            results_file.write('_'*10)
            results_file.write('\n')
    
    
    # STORE PARAMS
    with open(f'analysis_report_{analysis_name}/config.json', "w") as params_file:
        config_params = {'analysis_name': analysis_name, 
                         'model_name': model_name, 
                         'train_data_path': train_data_path, 
                         'val_data_path':   val_data_path,
                         'train_data_name': train_data_name,
                         'val_data_name': val_data_name,
                         'encoder_max_length': encoder_max_length, 
                         'decoder_max_length': decoder_max_length,
                         'random_state': random_state, 
                         'learning_rate': learning_rate,
                         'epochs': epochs, 
                         'train_batch': train_batch, 
                         'eval_batch': eval_batch,
                         'warmup_steps': warmup_steps, 
                         'weight_decay': weight_decay,
                         'logging_stes': logging_stes,
                         'save_total_lim': save_total_lim,
                         'label_smooting': label_smooting,
                         'predict_generate': predict_generate,
                         'eval_columns_list': eval_columns_list,
                         'save_strategy' : save_strategy,
                         }
        json.dump(config_params, params_file)

In [ ]:
'model_name': str, 
'train_data_path': str, 
'val_data_path':   str,
'train_data_name': str,
'val_data_name': str,
'encoder_max_length': int, 
'decoder_max_length': int,
'random_state': int, 
'learning_rate': float=5e-5,
'epochs': int=4, 
'train_batch': int=4, 
'eval_batch': int=4,
'save_strategy': ''
'load_best_model_at_end' : 
'evaluate_only'=False,
'report_to'=None

In [9]:
# model_name = '../5-domain-pre-training/analysis_report_CodeT5-test-12-300-4-2023-09-26-v3/results/checkpoint-78656'
# model_name = '../5-domain-pre-training/results/checkpoint-12290'
model_name = "CodeT5"
# model_name = '../4-qode2desc/no_bootstrap/analysis_report_CodeTrans_domain-checkpoint-12290-test-10-512-16-2023-10-07/results/checkpoint-2500'
# model_name = '../5-domain-pre-training/arxiv_pretraining/results/checkpoint-12290'

# model_name = "../5-domain-pre-training/analysis_report_CodeTrans-12-300-15-4-202310145_1/results/checkpoint-58500"
# model_name = "../5-domain-pre-training/analysis_report_CodeT5-4-300-15-4-202310145_1/results/checkpoint-19500"
# model_name = "../5-domain-pre-training/analysis_report_CodeT5-4-300-100-4-20231017_1/results/checkpoint-39000"


DATE = "20231021"

SAMPLE = "test"
if SAMPLE == "test":
    load_best_model_at_end = False
else:
    load_best_model_at_end = None

# tokenization
encoder_max_length = 512
decoder_max_length = 150
RS = 42
LR = 5e-5

EPOCHS = 3
TRAIN_BATCH = 16
EVAL_BATCH = 4

WARMUP_STEPS = 500
WEIGHT_DECAY = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 1
SAVE_STRATEGY = "steps"

LABEL_SMOOTHING = 0.1
PREDICT_GENERATE = True

MODE = "no_context"

EVAL_COLUMNS = [
    "eval_loss",
    "eval_rouge1",
    "eval_rouge2",
    "eval_rougeL",
    "eval_rougeLsum",
    "eval_bleu",
    "eval_gen_len",
]


analysis_name = (
    model_name
    + "-"
    + MODE
    + "-"
    + SAMPLE
    + "-"
    + str(EPOCHS)
    + "-"
    + str(encoder_max_length)
    + "-"
    + str(TRAIN_BATCH)
)

if len(model_name) > 15:
    analysis_name = (
        model_name.split("/")[-1]
        + "-"
        + SAMPLE
        + "-"
        + str(EPOCHS)
        + "-"
        + str(encoder_max_length)
        + "-"
        + str(TRAIN_BATCH)
    )

# analysis_name = "CodeT5_domain_4_19500"
analysis_name = analysis_name + "-" + DATE + "_" + str(RS)

print(analysis_name)

CodeT5-no_context-test-3-512-16-20231021_42


In [10]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [11]:
print(analysis_name)
analysis_modules.scs_analyze(
    analysis_name=analysis_name,
    model_name=model_name,
    train_data_path=f"../../data/preprocessed/Quantlet/{DATE}/{MODE}/",
    train_data_name=f"full_train_dataset_{DATE}_sample0.json",
    val_data_path=f"../../data/preprocessed/Quantlet/{DATE}/{MODE}/",
    val_data_name=f"test_dataset_{DATE}_sample0.json",
    encoder_max_length=encoder_max_length,
    decoder_max_length=decoder_max_length,
    random_state=RS,
    eval_columns_list=EVAL_COLUMNS,
    learning_rate=LR,
    epochs=EPOCHS,
    train_batch=TRAIN_BATCH,
    eval_batch=EVAL_BATCH,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    logging_stes=LOGGING_STEPS,
    save_total_lim=SAVE_TOTAL_LIM,
    save_strategy=SAVE_STRATEGY,
    label_smooting=LABEL_SMOOTHING,
    predict_generate=PREDICT_GENERATE,
    load_best_model_at_end=load_best_model_at_end,
)

CodeT5-no_context-test-3-512-16-20231021_42
CodeT5-no_context-test-3-512-16-20231021_42
cuda


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


cuda


Found cached dataset json (/home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-976d380dd9a2721a/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

...

/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,5.881600
200,5.109300
300,4.761600
400,4.429900
500,4.203100
600,3.861700
700,3.689300
800,3.671500


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      4.053        0.323        0.142        0.274           0.282   

   eval_bleu  eval_gen_len  
0      0.048        17.752  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


__________
Original: Algorithm that identify the S&P500 firm tickers from news text. The algorithm only identify those tickers in the three most commonly seen cases. Detailed description refer to the paper
Summary before Tuning: This function is used to create a single - leader version of a nltk object.
Summary after Tuning: This Quantlet is dedicated to finding the articles in the Marco website. The
__________


__________
Original: Inspect the derived variables from the News Network, Plots and Stats
Summary before Tuning: Generate a single - month
 from a CSV file.
Summary after Tuning: This Quantlet simulates the attention variables for the SPX article data set.
__________


__________
Original: Inspect the derived variables from the News Network, Plots and Stats
Summary before Tuning: A object for plotting a series of KDE with given bandwidths and kernel functions.
Summary after Tuning: Plotting Kernel Density  and Kernel Kernel Density 
__________


__________
Original: Estimates 

In [12]:
gc.collect()
torch.cuda.empty_cache()